In [8]:
import pmdarima as pm
import pandas as pd
import numpy as np
from pmdarima import model_selection
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

from statsmodels.tsa.stattools import acf, pacf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

from statsmodels.tsa.arima_model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
from sklearn.preprocessing import LabelEncoder
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import numpy as np


from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic
import warnings
warnings.filterwarnings('ignore')


In [5]:
from statsmodels.tsa.seasonal import STL

In [4]:
import os

In [ ]:
os.listdir('./')

In [25]:
from statsmodels.tsa.arima.model import ARIMA

In [10]:
data = pd.read_csv('../data/train.csv')

# Preprocess the data (assuming the date column is named 'date' and the value column is named 'value')
data['date'] = pd.to_datetime(data['date'])
data.set_index('date', inplace=True)

# Ensure the directory for saving models exists
#os.makedirs('models', exist_ok=True)

In [18]:
from pandas.plotting import autocorrelation_plot

In [ ]:
autocorrelation_plot(data['num_sold'])
pyplot.show()

## Import Data

In [8]:
train = pd.read_csv('../data/train.csv')
blind = pd.read_csv('../data/test.csv')

In [18]:
from statsmodels.tsa.stattools import adfuller

def check_stationarity(series, signif=0.05,  verbose=False):
    
    series.dropna(inplace = True)
    result = adfuller(series.values)

    print('ADF Statistic: %f' % result[0])
    print('p-value: %f' % result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t%s: %.3f' % (key, value))

    if (result[1] <= 0.05) & (result[4]['5%'] > result[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")
        

In [21]:
for country in data.country.unique():
    filtered = data[data['country'] == country]
    print(f'    Augmented Dickey-Fuller Test on "{country}"', "\n   ", '-'*47)

    check_stationarity(filtered[['num_sold']])
    print('\n')

    Augmented Dickey-Fuller Test on "0" 
    -----------------------------------------------
ADF Statistic: -16.960872
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Stationary


    Augmented Dickey-Fuller Test on "1" 
    -----------------------------------------------
ADF Statistic: -17.412301
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Stationary


    Augmented Dickey-Fuller Test on "2" 
    -----------------------------------------------
ADF Statistic: -16.064493
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Stationary


    Augmented Dickey-Fuller Test on "3" 
    -----------------------------------------------
ADF Statistic: -10.494190
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: -2.567
Stationary


    Augmented Dickey-Fuller Test on "4" 
    -----------------------------------------------
ADF Statistic: -10.662284
p-value: 0.000000
Critical Values:
	1%: -3.431
	5%: -2.862
	10%: 

train= train.dropna()

In [12]:
le = LabelEncoder()
data['country'] = le.fit_transform(data['country'])
data['store'] = le.fit_transform(data['store'])
data['product'] = le.fit_transform(data['product'])


In [22]:
# Model Evaluation 
data = data.dropna()
# Fit a simple auto_arima model
arima = pm.auto_arima(data['num_sold'],X=data.drop(columns=['num_sold']), d=2,error_action='ignore', trace=True,
                      suppress_warnings=True, maxiter=2,m=4,stationary=True,test='adf')

# Print out summary information on the fit
print(arima.summary())
print(arima.params)



pred=arima.predict(n_periods=100, X=test.drop(columns=['date','meantemp']), alpha=0.05)
pred

plt.figure(figsize=(15,5))
plt.grid()
#plt.plot(data['Period'][:-10], train['GDP_GROWTH'], marker='o', label='Train')
plt.plot(train['date'][-100:],bli['meantemp'], marker='o', label='Test')
plt.plot(train['date'][-100:],pred, marker='v', label='Prediction')
plt.legend()
plt.xticks(rotation=90)
plt.show()
print("="*50)
print("="*50)
mape=mean_absolute_percentage_error(test['meantemp'], pred)
print("Mean Absolute Percentage Error", mape)
print("="*50)
mse=mean_squared_error(test['meantemp'], pred)
print("Mean Squared Error",mse)
print("="*50)
print("RMSE", (mse)**0.5)
print("="*50)
print("AIC values {}".format(arima.aic()))
print("="*50)
print("="*50)
print("="*50)
print("="*50)


a=pd.DataFrame(test['meantemp'])
a=a.reset_index()
a=a.drop(columns='index')
a

prediction=pd.DataFrame(pred)
prediction=prediction.reset_index()
prediction=prediction.drop(columns='index')
prediction
output=pd.DataFrame()
output['Test']=a['meantemp']
output['Prediction']=prediction[0]
output['Difference']=output['Test']-output['Prediction']
output['% Chnage']=abs(output['Difference']/output['Test'])*100
output

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=3543444.748, Time=32.00 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=3510886.296, Time=34.77 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=3477199.268, Time=18.75 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=3463564.924, Time=19.99 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=6014262.827, Time=4.21 sec
 ARIMA(0,0,1)(0,0,0)[4] intercept   : AIC=3461879.060, Time=29.68 sec
 ARIMA(0,0,1)(1,0,0)[4] intercept   : AIC=3479523.030, Time=16.68 sec
 ARIMA(0,0,1)(1,0,1)[4] intercept   : AIC=3463594.167, Time=22.81 sec
 ARIMA(1,0,1)(0,0,0)[4] intercept   : AIC=3457573.047, Time=57.66 sec
 ARIMA(1,0,1)(1,0,0)[4] intercept   : AIC=3477328.645, Time=21.08 sec
 ARIMA(1,0,1)(0,0,1)[4] intercept   : AIC=3459436.909, Time=22.61 sec
 ARIMA(1,0,1)(1,0,1)[4] intercept   : AIC=3460055.761, Time=24.85 sec
 ARIMA(1,0,0)(0,0,0)[4] intercept   : AIC=3457573.392, Time=54.98 sec
 ARIMA(2,0,1)(0,0,0)[4] intercept   : AIC=351933

NameError: name 'test' is not defined

In [ ]:
os.makedirs('models', exist_ok=True)

# Fit ARIMA models for each category
categories = train['country'].unique()  # Assuming 'category' is the name of the categorical column
for category in categories:
    category_data = data[data['category'] == category]['value']
    
    # Fit the ARIMA model
    model = ARIMA(category_data, order=(5, 1, 0))  # Example order, adjust as necessary
    model_fit = model.fit()
    
    # Save the trained model
    joblib.dump(model_fit, f'models/trained_model_{category}.pkl')